In [4]:
import requests
import geopandas as gpd
import geojson
import pandas as pd


In [5]:
ames_housing = pd.read_csv('data/AmesHousing.csv')
realestate_data = pd.read_csv('data/Ames_Real_Estate_Data.csv')

neighbourhoods = pd.read_csv('data/neighbourhood_codes.csv', names=['codes', 'Neighborhood_name'])
neighbourhoods['codes']=neighbourhoods['codes'].str.strip()


C:\Users\local_x964246D\Temp\ipykernel_39120\812275049.py:2: DtypeWarning: Columns (36,40,81,87) have mixed types. Specify dtype option on import or set low_memory=False.
  realestate_data = pd.read_csv('data/Ames_Real_Estate_Data.csv')


In [6]:
cleaned_data = ames_housing.merge(neighbourhoods, left_on='Neighborhood', right_on='codes')

In [7]:
# Confirming no problems with the join
cleaned_data.loc[:,['Neighborhood', 'Neighborhood_name']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2487 entries, 0 to 2486
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Neighborhood       2487 non-null   object
 1   Neighborhood_name  2487 non-null   object
dtypes: object(2)
memory usage: 39.0+ KB


In [8]:
cleaned_data = cleaned_data.merge(realestate_data, left_on='PID', right_on='MapRefNo')

In [9]:
# Confirming no problems with the join
print(len(cleaned_data.loc[:,'PID'].unique()))
cleaned_data.loc[:,['PID', 'MapRefNo']]

2459


,PID,MapRefNo
0,527105010,527105010
1,527105030,527105030
2,527162130,527162130
3,527163010,527163010
4,527165230,527165230
...,...,...
2502,533350020,533350020
2503,532476050,532476050
2504,533250130,533250130
2505,533252020,533252020


In [15]:
import geocoder

# Aquiring json coordinates
cleaned_data['geolocation'] = cleaned_data['Prop_Addr_Full'].apply(lambda x: geocoder.arcgis(x).json)

In [ ]:
for i in range(len(cleaned_data['geolocation'])):
    cleaned_data['lat'][i] = cleaned_data['geolocation'][i]['lat']
    cleaned_data['lng'][i] = cleaned_data['geolocation'][i]['lng']


In [ ]:
# Remove entry with no address
cleaned_data = cleaned_data[~(cleaned_data['PID']==533210020)]

cleaned_data['price_sq2'] = cleaned_data['SalePrice_x']/cleaned_data['Lot Area']

cleaned_data.to_csv('data/geodata.csv')

In [ ]:
cleaned_data['geometry'] = gpd.points_from_xy(cleaned_data.lng, cleaned_data.lat, crs="EPSG:4326")

In [ ]:
cleaned_data_gdf = gpd.GeoDataFrame(cleaned_data, geometry='geometry')

In [ ]:
display(cleaned_data.drop(columns='geometry'))

Databricks data profile. Run in Databricks to view.

In [ ]:
cleaned_data_gdf.dissolve().centroid

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt

mapbox_access_token = "pk.eyJ1IjoiZnJlZGVyaWNvcm9kcmlndWVzIiwiYSI6ImNrbmp6OGEyYjA2YWQycXBuYzc0c3Bha3oifQ.yQLJy408VtnzKC0Vgijv8A"

selected_column='Neighborhood_name'
gdf = cleaned_data_gdf
# Get all unique values from the 'category_column'
gdf = gdf.sort_values(by=selected_column)
unique_values = gdf.loc[:,selected_column].unique()

# Generate a list of distinct colors using the 'tab20' colormap from matplotlib
colors = plt.cm.tab20.colors
color_scale = [f'rgb({int(r * 255)},{int(g * 255)},{int(b * 255)})' for r, g, b in colors]
# Create the 'category_colors' dictionary by mapping each unique category to a color
unique_values_colours = {category: color for category, color in zip(unique_values, color_scale)}

# Map the categories in the GeoDataFrame to the corresponding colors
gdf['color'] = gdf[selected_column].map(unique_values_colours)

fig = go.Figure()

for val in unique_values:
    df_color = gdf[gdf[selected_column] == val]

    fig.add_trace(
        go.Scattermapbox(
                        name=val,
                        lat=df_color['lat'],
                        lon=df_color['lng'],
                        mode = 'markers',
                        marker = dict(
                            size = 5,
                            opacity = 1,

                            color = df_color['color'],
                        ),
                        # hovertext=df_color['tooltip'],
                        hovertemplate='<extra></extra>',
                    )
                )

fig.update_layout(
   autosize=True,
   hovermode='closest',
   mapbox=dict(
       accesstoken=mapbox_access_token,
       bearing=0,
       center=dict(
           lat=42.03327,
           lon=-93.64773
       ),
       pitch=0,
       zoom=10
   ),
)

display(fig)

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt

mapbox_access_token = "pk.eyJ1IjoiZnJlZGVyaWNvcm9kcmlndWVzIiwiYSI6ImNrbmp6OGEyYjA2YWQycXBuYzc0c3Bha3oifQ.yQLJy408VtnzKC0Vgijv8A"

selected_column='price_sq2'
gdf = cleaned_data

gdf.dropna(subset=[selected_column], inplace=True)

fig = go.Figure()

fig.add_trace(
    go.Scattermapbox(
                    name=val,
                    lat=gdf['lat'],
                    lon=gdf['lng'],
                    mode = 'markers',
                    marker = dict(
                        size = 5,
                        opacity = 1,
                        color=gdf[selected_column],
                        colorscale='RdYlGn',
                        colorbar=dict(
                            title="Price per sq feet",
                            thicknessmode="pixels", thickness=50,
                            # lenmode="pixels", len=200,
                            yanchor="top", y=1,
                            ticks="outside",
                            dtick=5
                        )
                    ),
                    hovertext=gdf[selected_column],
                    # hovertemplate='<extra></extra>',
                )
            )

fig.update_layout(
   autosize=True,
   hovermode='closest',
   mapbox=dict(
       accesstoken=mapbox_access_token,
       bearing=0,
       center=dict(
           lat=42.03327,
           lon=-93.64773
       ),
       pitch=0,
       zoom=10
   ),
)

display(fig)

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt

mapbox_access_token = "pk.eyJ1IjoiZnJlZGVyaWNvcm9kcmlndWVzIiwiYSI6ImNrbmp6OGEyYjA2YWQycXBuYzc0c3Bha3oifQ.yQLJy408VtnzKC0Vgijv8A"

selected_column='Neighborhood_name'
gdf = cleaned_data_gdf

values = cleaned_data_gdf['Lot Area']
max_value = values.max()

def color_scale(value):
   return 'rgb(0, 0, {})'.format(int(255 * value / max_value))

# Create a Scattermapbox object
scatter = go.Scattermapbox(
    lat=df_color['lat'],
    lon=df_color['lng'],
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = [10],
        color = [color_scale(value) for value in values], # set color based on a data column
        opacity = [0.8],
    ),
    text = ['Marker 1'],
)

fig = go.Figure(data=scatter)

fig.update_layout(
   autosize=True,
   hovermode='closest',
   mapbox=dict(
       accesstoken=mapbox_access_token,
       bearing=0,
       center=dict(
           lat=42.03327,
           lon=-93.64773
       ),
       pitch=0,
       zoom=10
   ),
)

display(fig)